# Построения РС Popularity-based model

Для начала загрузим датасет "Articles sharing and reading from CI&T DeskDrop", включающий в себя собранные за один год логи DeskDrop — платформы для внутренних коммуникаций, разработанной CI&T и ориентированной на компании, использующие Google Workspace (Google G Suite). Среди прочего, эта платформа позволяет сотрудникам компаний делиться актуальными статьями со своими коллегами.

В датасете содержится около 73 тысяч записей о взаимодействии пользователей с более чем тремя тысячами публичных статей, размещённых на платформе.

Информация в наборе данных:
- Оригинальный URL, название и текст статьи.
- Контекст посещений пользователей, например дата/время, клиент (мобильное приложение/браузер) и геолокация.
- Различные типы взаимодействия, что позволяет сделать вывод об уровне заинтересованности пользователя в статьях, например комментарии → лайки → просмотры.

Данные включают в себя два файла:
- shared_articles.csv;
- users_interactions.csv.

Начнём работать с файлом shared_articles.csv. Он содержит информацию о статьях, опубликованных на платформе DeskDrop.

Для каждой статьи есть:
- дата публикации (временная метка),
- исходный URL-адрес,
- заголовок,
- содержание в виде обычного текста,
- язык статьи (португальский — pt или английский — en),
- информация о пользователе, который поделился статьёй (автор).

Для временной метки существует два возможных типа событий:
- CONTENT SHARED — статья была опубликована на платформе и доступна для пользователей;
- CONTENT REMOVED — статья была удалена с платформы и недоступна для дальнейших рекомендаций.

Для простоты мы рассматриваем здесь только тип события CONTENT SHARED.

In [3]:
import pandas as pd
import numpy as np

shared_articles = pd.read_csv('data/shared_articles.csv')
shared_articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


Задание 6.1

Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?

In [5]:
content_shared = shared_articles[shared_articles['eventType'] == 'CONTENT SHARED'] 
content_shared.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3047 entries, 1 to 3121
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        3047 non-null   int64 
 1   eventType        3047 non-null   object
 2   contentId        3047 non-null   int64 
 3   authorPersonId   3047 non-null   int64 
 4   authorSessionId  3047 non-null   int64 
 5   authorUserAgent  669 non-null    object
 6   authorRegion     669 non-null    object
 7   authorCountry    669 non-null    object
 8   contentType      3047 non-null   object
 9   url              3047 non-null   object
 10  title            3047 non-null   object
 11  text             3047 non-null   object
 12  lang             3047 non-null   object
dtypes: int64(4), object(9)
memory usage: 333.3+ KB


Теперь откроем второй файл — users_interactions.csv.

In [39]:
import pandas as pd
import numpy as np

users_interactions = pd.read_csv('data/users_interactions.csv')
users_interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


Давайте предварительно преобразуем столбцы personId, contentId в таблицах к строкам. Это преобразование пригодится нам в дальнейшем:


In [40]:
users_interactions.personId = users_interactions.personId.astype(str)
users_interactions.contentId = users_interactions.contentId.astype(str)
content_shared.contentId = content_shared.contentId.astype(str)

C:\Users\alexD\AppData\Local\Temp\ipykernel_17748\157489056.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_shared.contentId = content_shared.contentId.astype(str)


В колонке eventType описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:
- VIEW — просмотр,
- LIKE — лайк,
- COMMENT CREATED — комментарий,
- FOLLOW — подписка,
- BOOKMARK — добавление в закладки.

В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:

In [23]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

Веса здесь подобраны исходя из важности каждого действия: оставить комментарий — значит, показать наибольшую вовлечённость, а обычный просмотр, напротив, демонстрирует наименьшую вовлечённость.

Задание 6.2

Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

In [42]:
# Словарь с весами типа события
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0,
   'BOOKMARK': 2.5,
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

# Переименовываем столбец eventType в event_type (если нужно)
users_interactions.rename(columns={'eventType': 'event_type'}, inplace=True)

# Добавляем столбец с весами взаимодействий
users_interactions['interaction_weight'] = users_interactions['event_type'].map(event_type)

# Вычисляем среднее значение interaction_weight и округляем до 2 знаков
average_weight = round(users_interactions['interaction_weight'].mean(), 2)

# Выводим результат
print(f"Среднее значение веса взаимодействия: {average_weight}")

Среднее значение веса взаимодействия: 1.24


Ранее мы говорили, что рекомендательные системы подвержены проблеме холодного старта — в таких случаях создавать рекомендации намного сложнее.

Задание 6.3

Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

In [51]:
user_article_counts = (
    users_interactions
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions = \
    user_article_counts[user_article_counts >= 5].reset_index()[['personId']]

print(len(users_with_enough_interactions))

1140


Задание 6.4

Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

В этом задании необходимо работать с данными, полученными в задании 6.3.

In [52]:
interactions_from_selected_users = users_interactions.loc[np.in1d(users_interactions.personId,
            users_with_enough_interactions)]
print(interactions_from_selected_users.shape)

(69868, 9)


Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

In [56]:
import math

def smooth_user_preference(x):
    return math.log(1+x, 2)

Задание 6.5

Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

Так как наши данные отсортированы по дате, то для того, чтобы выбрать последнее взаимодействие, необходимо использовать метод max().

В этом задании необходимо работать с данными, полученными в задании 6.4.

In [57]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full = (
    interactions_from_selected_users
    .groupby(['personId', 'contentId']).interaction_weight.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full['last_timestamp'] = (
    interactions_from_selected_users
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full = interactions_full.reset_index()
interactions_full['last_timestamp'].mean()

1470605340.0403006

Разумеется, для того чтобы впоследствии оценить качество построенной рекомендательной системы, нам нужно разделить выборку на обучающую и тестовую. Так как в реальности рекомендации строятся на основе исторических данных о пользователе и контенте, сделаем в нашей задаче разбиение на обучающую и тестовую выборки по временной отсечке.

Задание 6.6

Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Сколько объектов попало в обучающую выборку?

In [58]:
split_ts = 1475519545
interactions_train_df = interactions_full.loc[interactions_full.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full.loc[interactions_full.last_timestamp >= split_ts].copy()

print(len(interactions_train_df))

29325


Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.

In [59]:
final_df = (
    interactions_train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    interactions_test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


Задание 6.7

Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [60]:
popular = (
    interactions_train_df
    .groupby('contentId')
    .interaction_weight.sum().reset_index()
    .sort_values('interaction_weight', ascending=False)
    .contentId.values
)
popular[0]

'-6783772548752091658'

Теперь необходимо сформировать рекомендации для каждого пользователя. Будем рекомендовать десять самых популярных статей. Также необходимо помнить, что следует предлагать пользователю только то, что он ещё не читал.

Для вычисления precision@10 воспользуйтесь следующей функцией:

Задание 6.8

Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.

In [61]:
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()

Итоговый результат округлите до трёх знаков после точки-разделителя.

В этом задании необходимо использовать данные, полученные в задании 6.6 (для которых приведён скриншот выше).

In [62]:
top_k = 10
 
final_df['popular'] = (final_df.true_train.apply(lambda x: popular[~np.in1d(popular, x)][:top_k]))

def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()
calc_precision('popular')

0.006454207722621089

Качество получилось не очень высоким, но ведь и рекомендации у нас были неперсонализированными.

Примечание. Стоит отметить, что качество РС оценивается не так, как в задачах классификации: показателей выше 0.5 добиться практически невозможно, и даже результат 0.1–0.2 — индикатор высокого качества.

